In [1]:
# Python 샘플 코드 #
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [2]:
# 해당 코드에서 크롤링을 시도할 연도(yyyy00)
this_year = 200800

In [3]:
'''
국토교통부_아파트매매 실거래 상세 자료
활용기간 2023.04.12 까지
LAWD_CD = 11110  # 47290(경산) 지역코드 - 각 지역별 코드 행정표준코드관리시스템(www.code.go.kr)의 법정동코드 10자리 중 앞 5자리
DEAL_YMD = 200601  # 계약월 - 실거래 자료의 계약년월 6자리
numOfRows = 500  # 한 페이지 결과 수
pageNo = 1  # 페이지번호
'''
def get_apt_data(LAWD_CD, DEAL_YMD, serviceKey, numOfRows=10000):
    # 호출하려는 OpenAPI URL 정의
    data_url = "".join(
        ["http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?",
         "serviceKey=", serviceKey, "&LAWD_CD=", str(LAWD_CD), "&DEAL_YMD=", str(DEAL_YMD), "&numOfRows=",
         str(numOfRows)])

    # 정의된 OpenAPI URL을 호출
    data_response = requests.get(data_url)
    data_xml = BeautifulSoup(data_response.text, "xml")
    data_xml_row = data_xml.find_all("item")
    data_xml_row

    # XML에 포함된 항목들을 리스트로 저장
    list_items = ["거래금액", "건축년도", "년", "도로명", "도로명건물본번호코드", "도로명건물부번호코드",
                  "도로명시군구코드", "도로명일련번호코드", "도로명지상지하코드", "도로명코드",
                  "법정동", "법정동본번코드", "법정동부번코드", "법정동시군구코드", "법정동읍면동코드", "법정동지번코드",
                  "아파트", "월", "일", "일련번호", "전용면적", "지번", "지역코드", "층", "해제사유발생일", "해제여부"]

    # 리스트의 리스트 형태로 데이터프레임에 저장할 자료 만들기
    list_dataframe = []

    for n_obs in range(0, len(data_xml_row)):
        list_obs_values = []  # 동일한 관측치의 데이터는 모두 이 리스트에 저장된다
        for i in range(0, len(list_items)):
            # 특정 항목이 XML에 아예 존재하지 않는 경우가 있으므로 try 구문 사용
            try:
                list_obs_values.append(data_xml_row[n_obs].find(list_items[i]).text)
            except Exception as e:
                list_obs_values.append("")
                print("Error in " + str(n_obs) + "th obs, item: " + list_items[i] + ", " + str(e))
        list_dataframe.append(list_obs_values)  # 리스트의 리스트로 관측치 저장

    # 데이터프레임 형태로 변환
    df_apt = pd.DataFrame(list_dataframe, columns=list_items)
    return df_apt

In [4]:
# 함수 작동 여부 확인
serviceKey_YKim = "%2F2a7hudNIqdsS1Uq7SaicUQYvpno8zZ4M53iUr4PgZnXIDq4SMhbDLNHrk6PfFXNdnb7LuNpEffhq8YXRq2dbQ%3D%3D"
df_tmp1 = get_apt_data(LAWD_CD=11110, DEAL_YMD=202102, serviceKey=serviceKey_YKim)

Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 18th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 19th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 20th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 21th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 25th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 일련번호, 'NoneType' object has no attribute 'text'
Error in 36th obs, item: 일련번호, 'NoneType' object has no attribute 'text'


In [5]:
# Import TXT file 법정동코드 전체자료 불러오기
#df_lawd_cd = pd.read_csv('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
#paper_prepare/data_raw/법정동코드 전체자료.txt', sep="\t", engine='python', encoding="CP949")
df_lawd_cd = pd.read_csv('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/법정동코드 전체자료.txt', sep="\t", engine='python', encoding="CP949")
df_lawd_cd["LAWD_CD"] = df_lawd_cd["법정동코드"].divmod(100000)[0]
df_lawd_cd_nodup = df_lawd_cd.drop_duplicates(subset=['LAWD_CD']).copy()  # 중복값 제외
df_lawd_cd_nodup_exist = df_lawd_cd_nodup[df_lawd_cd_nodup["폐지여부"] == "존재"].copy()

In [6]:
total_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"]
tmp_lawd_cd = df_lawd_cd_nodup_exist["LAWD_CD"][3:10]

In [7]:
# 국토교통부_아파트매매 실거래 상세 자료는 2008년 1월부터 자료가 존재한다.
# 그런데 2006년으로 입력해도 자료가 있다.

In [8]:

# 시작 시점과 종료 시점 입력
yyyymm_start = this_year + 1  # 시작 년월
yyyymm_end = this_year + 3  # 종료 년월

list_yyyymm = []
for n_yyyymm in range(yyyymm_start, yyyymm_end+1):
    if (divmod(n_yyyymm, 100)[1] != 0) & (divmod(n_yyyymm, 100)[1] < 13):
        list_yyyymm.append(n_yyyymm)

list_yyyymm.reverse()


# StopWatch: 코드 시작
time_this_code_start = datetime.now()
print("This code started at: " + str(time_this_code_start))

# DataFrame 구성을 위해 공백 자료를 하나 만든다.
df_dataset_null = get_apt_data(LAWD_CD=11120, DEAL_YMD=202102, serviceKey=serviceKey_YKim)

# 전국의 한달 자료 받는데 약 9분 소요
for yyyymm in list_yyyymm:
    df_dataset = df_dataset_null.copy()
    for lawd_cd in total_lawd_cd:
        df_apt = get_apt_data(LAWD_CD=lawd_cd, DEAL_YMD=yyyymm, serviceKey=serviceKey_YKim)
        # df_apt.to_pickle("./data_raw/df_apt_" + str(yyyymm) + "_" + str(lawd_cd) + ".pkl")
        df_dataset = pd.concat([df_dataset, df_apt])
    df_dataset.to_pickle("C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_" + str(yyyymm) + ".pkl")

# StopWatch: 코드 종료
time_this_code_end = datetime.now()
print("This code finished at: " + str(time_this_code_end))
print("Elapsed (in this code): " + str(time_this_code_end - time_this_code_start))

This code started at: 2021-05-20 02:21:06.722431
Error in 60th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 61th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 61th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 61th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 61th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 61th obs, item: 도로명일련번호코드, 'NoneType' object has no attribute 'text'
Error in 61th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 61th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 62th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 47th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 54th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 57th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 63th obs, item:

Error in 979th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1000th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1022th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1023th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1069th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1111th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1155th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1250th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1270th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1291th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1317th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 1368th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 53th obs, item: 도로명지상지하코드, 'None

Error in 93th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 100th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 100th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 103th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 103th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 141th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 149th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 156th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 174th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 175th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 192th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 193th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 89th obs, item: 도로명건물본번호코드, 'NoneType' o

Error in 253th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 253th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 253th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 253th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 253th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 264th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 264th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 264th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 264th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 264th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 265th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 265th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 265th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 't

Error in 123th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 128th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 132th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 142th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 151th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 155th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 164th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 165th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 166th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 171th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 177th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 179th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 184th obs, item: 도로명지상지하코드, 'NoneType' obje

...

Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 24th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 31th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 35th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 50th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 65th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 82th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 85th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 87th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 88th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 92th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 95th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 97th obs, item: 도로명지상지하코드, 'NoneType' object has no att

Error in 56th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 56th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 56th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 56th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 56th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 65th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 65th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 65th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in 65th obs, item: 도로명시군구코드, 'NoneType' object has no attribute 'text'
Error in 65th obs, item: 도로명코드, 'NoneType' object has no attribute 'text'
Error in 66th obs, item: 도로명, 'NoneType' object has no attribute 'text'
Error in 66th obs, item: 도로명건물본번호코드, 'NoneType' object has no attribute 'text'
Error in 66th obs, item: 도로명건물부번호코드, 'NoneType' object has no attribute 'text'
Error in

Error in 2th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 3th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 5th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 6th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 0th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 10th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 11th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 12th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 13th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 14th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 15th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 16th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 17th obs, item: 도로명지상지하코드, 'NoneType' object has no attribut

Error in 197th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 201th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 202th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 204th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 205th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 206th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 207th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 208th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 209th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 210th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 211th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 212th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 217th obs, item: 도로명지상지하코드, 'NoneType' obje

Error in 7th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 0th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 8th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 12th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 69th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 73th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 84th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 57th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 59th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 61th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 78th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 85th obs, item: 도로명지상지하코드, 'NoneType' object has no attribute 'text'
Error in 86th obs, item: 도로명지상지하코드, 'NoneType' object has no attrib

In [9]:
# Check the data

data1 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start)+'.pkl')

data2 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 1)+'.pkl')

data3 = pd.read_pickle('C:/Users/Kimyihwan/OneDrive - kaist.ac.kr/\\\
paper_prepare/data_raw/df_dataset_'+str(yyyymm_start + 2)+'.pkl')

In [10]:
data1

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"37,500",2002,2008,필운대로10길,00016,00000,11110,01,0,4100521,...,신현(101동),1,25,11110-7,84.82,6-13,11110,2,,
1,"20,000",1973,2008,사직로,00095,00000,11110,02,0,3100005,...,사직파크맨션,1,10,11110-211,99.17,142,11110,6,,
2,"38,500",2007,2008,필운대로2길,00002,00000,11110,03,0,4100525,...,두레엘리시안,1,10,11110-2175,84.74,174-1,11110,6,,
3,"118,000",2003,2008,사직로8길,00020,00000,11110,05,0,4100135,...,경희궁 파크팰리스,1,15,11110-107,146.39,95,11110,15,,
4,"120,000",2004,2008,경희궁2길,00005,00005,11110,01,0,4100005,...,킹스매너,1,31,11110-118,194.43,110-15,11110,3,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,"5,200",2000,2008,최남단해안로109번길,00070,00000,50130,01,0,4850859,...,흥도빌라트,1,10,50130-68,78.53,430-1,50130,4,,
60,"5,120",2004,2008,신영로,00065,00011,50130,01,0,3350186,...,에스빌라트,1,18,50130-81,84.83,1073-3,50130,6,,
61,"3,520",2007,2008,동일하모로,00188,00000,50130,01,0,3350067,...,유성아트빌,1,24,50130-538,37.8,1440-6,50130,5,,
62,"7,200",2007,2008,동일하모로,00188,00000,50130,01,0,3350067,...,유성아트빌,1,25,50130-538,79.81,1440-6,50130,4,,


In [11]:
data2

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"11,300",1999,2008,옥인3길,00007,00000,11110,01,0,4100207,...,청호그린빌,2,4,11110-210,56.4,40,11110,5,,
1,"10,000",1999,2008,옥인3길,00007,00000,11110,01,0,4100207,...,청호그린빌,2,20,11110-210,56.4,40,11110,1,,
2,"110,000",2003,2008,사직로8길,00020,00000,11110,05,0,4100135,...,경희궁 파크팰리스,2,1,11110-107,146.33,95,11110,3,,
3,"22,577",2005,2008,율곡로2길,00007,00000,11110,01,0,4100234,...,로얄팰리스스위트,2,22,11110-205,36.86,85,11110,15,,
4,"15,500",2003,2008,돈화문로11가길,00059,00000,11110,03,0,4100050,...,현대뜨레비앙,2,21,11110-102,46.64,55,11110,5,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,"4,600",2002,2008,지세못로,00029,00000,50130,01,0,3350255,...,서진,2,21,50130-67,49.8737,367-1,50130,1,,
29,"3,700",2000,2008,비행장로55번길,00016,00000,50130,01,0,4850299,...,한빛테크,2,25,50130-35,58.944,4148-1,50130,1,,
30,"7,750",2007,2008,동일하모로,00188,00000,50130,01,0,3350067,...,유성아트빌,2,15,50130-538,80.66,1440-6,50130,5,,
31,"3,550",1996,2008,남원광지로,00005,00000,50130,01,0,3350031,...,에덴,2,21,50130-44,40,1323-2,50130,2,,


In [12]:
data3

,거래금액,건축년도,년,도로명,도로명건물본번호코드,도로명건물부번호코드,도로명시군구코드,도로명일련번호코드,도로명지상지하코드,도로명코드,...,아파트,월,일,일련번호,전용면적,지번,지역코드,층,해제사유발생일,해제여부
0,"39,300",2002,2008,필운대로10길,00018,00008,11110,01,0,4100521,...,신현(102동),3,16,11110-112,84.82,6-11,11110,2,,
1,"108,000",2004,2008,사직로8길,00034,00000,11110,05,0,4100135,...,경희궁의아침3단지,3,13,11110-116,124.17,72,11110,3,,
2,"19,000",2005,2008,율곡로2길,00007,00000,11110,01,0,4100234,...,로얄팰리스스위트,3,14,11110-205,40.07,85,11110,10,,
3,"24,150",2005,2008,율곡로2길,00007,00000,11110,01,0,4100234,...,로얄팰리스스위트,3,17,11110-205,39.67,85,11110,13,,
4,"15,700",2003,2008,돈화문로11가길,00059,00000,11110,03,0,4100050,...,현대뜨레비앙,3,5,11110-102,46.64,55,11110,2,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,"4,300",2002,2008,지세못로,00029,00000,50130,01,0,3350255,...,서진,3,10,50130-67,49.8737,367-1,50130,1,,
29,"4,300",2002,2008,지세못로,00029,00000,50130,01,0,3350255,...,서진,3,31,50130-67,49.8737,367-1,50130,3,,
30,"4,300",2002,2008,지세못로,00029,00000,50130,01,0,3350255,...,서진,3,31,50130-67,49.8737,367-1,50130,4,,
31,"3,250",1991,2008,고성동서로56번길,00010,00000,50130,01,0,4850040,...,동명,3,6,50130-45,55.11,1132-21,50130,2,,
